In [7]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [8]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

In [9]:
train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48, 64]

test_lulesh_processes: list[int] = [512]
test_lulesh_iterations: list[int] = [256]
test_lulesh_sizes: list[int] = [128]

input_list_expVarNames: list[str] = ["process", "iteration", "size"]
input_list_resVarNames: list[str] = ["#Call"]

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelLin"
    # "modelBasicTree",
]

input_rawDF_train: pd.DataFrame = return_rawDFinLULESH(
    processes=train_lulesh_processes,
    iterations=train_lulesh_iterations,
    sizes=train_lulesh_sizes,
    csvDirPath=csvDirPath,
)
input_rawDF_test: pd.DataFrame = return_rawDFinLULESH(
    processes=test_lulesh_processes,
    iterations=test_lulesh_iterations,
    sizes=test_lulesh_sizes,
    csvDirPath=csvDirPath,
)

In [10]:
input_rawDF: pd.DataFrame = return_rawDFinLULESH(
    processes=lulesh_processes,
    iterations=lulesh_iterations,
    sizes=lulesh_sizes,
    csvDirPath=csvDirPath,
)

input_rawDF = input_rawDF.rename(columns={"Name": "functionName"})

input_list_expVarNames: list[str] = ["process", "iteration", "size"]
input_list_resVarNames: list[str] = ["#Call"]

mapeTableInLULESH: pd.DataFrame = returnMAPEtableInLULESH(
    rawDF=input_rawDF,
    list_expVar=input_list_expVarNames,
    list_resVar=input_list_resVarNames,
    list_modelName=list_modelName,
)

mapeTableInLULESH

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estima

,modelLin,modelIp,modelLog,modelProcessDividedByProblemSize,modelProblemSizeDividedByProcess,modelLinAndIp,modelLinAndLog,modelIpAndLin,modelIpAndLog,modelLogAndLin,modelLogAndIp,modelLinearSumOf2elementCombination,modelLinearSumOfElementCombinations,modelLinearSumOf2elementCombinationWithSquared,modelLinearSumOf2elementCombinationWithCubed,functionName
0,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,void_Domain::~Domain()
1,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,void_Domain::BuildMesh(Int_t_Int_t_Int_t)
2,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,void_Domain::Domain(Int_t_Index_t_Index_t_Inde...
3,1.500754e-13,172.922882,111.693928,8.499044e+01,7.624396e+01,172.922882,111.693928,7.287886e-14,111.693928,1.520599e-13,172.922882,113.765030,1.625247e-12,138.327962,138.327962,MPI_Waitall()
4,1.799660e-13,192.995792,127.512160,7.808887e+01,7.752953e+01,192.995792,127.512160,9.971133e-15,127.512160,4.489668e-14,192.995792,128.765533,2.410129e-12,157.504072,157.504072,MPI_Allreduce()
5,5.597246e+01,193.662948,151.023292,8.183808e+01,8.405442e+01,193.166473,142.326222,6.802352e+01,148.908970,7.353575e+01,192.710753,104.783416,3.866607e+00,142.235445,142.235445,MPI_Isend()
6,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,MPI_Finalize()
7,5.597246e+01,193.662948,151.023292,8.183808e+01,8.405442e+01,193.166473,142.326222,6.802352e+01,148.908970,7.353575e+01,192.710753,104.783416,3.866607e+00,142.235445,142.235445,MPI_Irecv()
8,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,void_Domain::CreateRegionIndexSets(Int_t_Int_t)
9,1.646390e-13,177.412047,115.178571,7.668323e+01,7.655747e+01,177.412047,115.178571,9.075633e-15,115.178571,4.070818e-14,177.412047,117.100765,2.199475e-12,142.581880,142.581880,void_CommMonoQ(Domain


In [11]:
mapeTableInLULESHwithLowestMAPE :pd.DataFrame = addLowestMAPEColumn(inputDF=mapeTableInLULESH, model_name_list=list_modelName, version=2)

mapeTableInLULESHwithLowestMAPE

,modelLin,modelIp,modelLog,modelProcessDividedByProblemSize,modelProblemSizeDividedByProcess,modelLinAndIp,modelLinAndLog,modelIpAndLin,modelIpAndLog,modelLogAndLin,modelLogAndIp,modelLinearSumOf2elementCombination,modelLinearSumOfElementCombinations,modelLinearSumOf2elementCombinationWithSquared,modelLinearSumOf2elementCombinationWithCubed,functionName,最低値
0,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,void_Domain::~Domain(),0.000000e+00
1,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,void_Domain::BuildMesh(Int_t_Int_t_Int_t),0.000000e+00
2,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,void_Domain::Domain(Int_t_Index_t_Index_t_Inde...,0.000000e+00
3,1.500754e-13,172.922882,111.693928,8.499044e+01,7.624396e+01,172.922882,111.693928,7.287886e-14,111.693928,1.520599e-13,172.922882,113.765030,1.625247e-12,138.327962,138.327962,MPI_Waitall(),7.287886e-14
4,1.799660e-13,192.995792,127.512160,7.808887e+01,7.752953e+01,192.995792,127.512160,9.971133e-15,127.512160,4.489668e-14,192.995792,128.765533,2.410129e-12,157.504072,157.504072,MPI_Allreduce(),9.971133e-15
5,5.597246e+01,193.662948,151.023292,8.183808e+01,8.405442e+01,193.166473,142.326222,6.802352e+01,148.908970,7.353575e+01,192.710753,104.783416,3.866607e+00,142.235445,142.235445,MPI_Isend(),3.866607e+00
6,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,MPI_Finalize(),0.000000e+00
7,5.597246e+01,193.662948,151.023292,8.183808e+01,8.405442e+01,193.166473,142.326222,6.802352e+01,148.908970,7.353575e+01,192.710753,104.783416,3.866607e+00,142.235445,142.235445,MPI_Irecv(),3.866607e+00
8,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,void_Domain::CreateRegionIndexSets(Int_t_Int_t),0.000000e+00
9,1.646390e-13,177.412047,115.178571,7.668323e+01,7.655747e+01,177.412047,115.178571,9.075633e-15,115.178571,4.070818e-14,177.412047,117.100765,2.199475e-12,142.581880,142.581880,void_CommMonoQ(Domain,9.075633e-15


In [12]:
mapeTableInLULESHwithLowestMAPE.mean()

/tmp/ipykernel_4767/1383311857.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mapeTableInLULESHwithLowestMAPE.mean()


modelLin                                          324.173667
modelIp                                           411.497704
modelLog                                          454.526334
modelProcessDividedByProblemSize                   40.081234
modelProblemSizeDividedByProcess                   35.645728
modelLinAndIp                                     421.093841
modelLinAndLog                                    455.735629
modelIpAndLin                                     315.660772
modelIpAndLog                                     446.710080
modelLogAndLin                                    323.795580
modelLogAndIp                                     419.026472
modelLinearSumOf2elementCombination               194.468150
modelLinearSumOfElementCombinations                41.843775
modelLinearSumOf2elementCombinationWithSquared    102.307726
modelLinearSumOf2elementCombinationWithCubed      102.307726
最低値                                                 4.756974
dtype: float64

***

---

___

# 2022年4月17日～

次のような表を作成する

採用される割合 (MAPE の最大値 [%] ，MAPE の最小値 [%]) [%]

| ベンチマークプログラム名 | 線形モデル               | 対数モデル               | 反比例モデル              |
|--------------|---------------------|---------------------|---------------------|
| str          | float(float, float) | float(float, float) | float(float, float) |


目標となるのは一気にこのベンチマークプログラムを作成することだが、既存のライブラリ関数などを利用し、まずはベンチマークごとに作成可能にする。

メモ

## 実装予定

1. 行方向に最小値を検出
2. 最小値以外をNaNに変更
3. 列方向に最小値と最大値を検出

## 


***

---

___